In [493]:
# Imports
import numpy as np
import pandas as pd
import polars as pl
import pulp
from tqdm.auto import tqdm
import os
import gc
import ast
import pickle
from warnings import filterwarnings
from variables import df0, master_availability, master_demand, decision_df, ddf, demand_decision_df, moq_decision_df, time_decision_df, cost_decision_df, cost_decision_df2, emission_decision_df, choice_decision, trip_decision, obj_df

filterwarnings("ignore")

# Load master data
vehicles_df = pd.read_csv("../data/Master_data/vehicle_master_data_4.csv")
distances_df = pd.read_csv("../data/Master_data/distance_master_data_4.csv")

# Read input
input_df = pd.read_csv("../data/Input_data/Input_1_Small.csv")

In [494]:
# ***** !!! *****
input_df["From"] = input_df["From"].str.strip()
input_df["To"] = input_df["To"].str.strip()
distances_df["Source"] = distances_df["Source"].str.strip()
distances_df["Destination"] = distances_df["Destination"].str.strip()

input_df["From"] = input_df["From"].str.replace(" ", "_")
input_df["To"] = input_df["To"].str.replace(" ", "_")
distances_df["Source"] = distances_df["Source"].str.replace(" ", "_")
distances_df["Destination"] = distances_df["Destination"].str.replace(" ", "_")

In [495]:
distances_df["Destination"].unique()

array(['Delhi', 'Bengaluru', 'Hyderabad', 'Ahmadabad', 'Surat', 'Kolkata',
       'Chennai', 'Jaipur', 'Pune', 'Kanpur', 'Lucknow', 'Nagpur',
       'Indore', 'Thane', 'Pimpri_Chinchwad', 'Bhopal', 'Patna',
       'Visakhapatnam', 'Ludhiana', 'Vadodara', 'Agra', 'Nashik',
       'Faridabad', 'Meerut', 'Rajkot', 'Vasai_Virar_City', 'Varanasi',
       'Srinagar', 'Dhanbad', 'Mumbai'], dtype=object)

In [496]:
def clean_data_mode(df):
    # Drop mode distance that are not compatible to the specific vehicle
    df.drop(
        df[(df["Air Distance (km)"].isna()) & (df["Transportation modes"] == "Air")].index,
        axis=0,
        inplace=True,
    )
    df.drop(
        df[(df["Rail Distance (km)"].isna()) & (df["Transportation modes"] == "Rail")].index,
        axis=0,
        inplace=True,
    )
    df.drop(
        df[(df["Road Distance (km)"].isna()) & (df["Transportation modes"] == "Road")].index,
        axis=0,
        inplace=True,
    )
    df.reset_index(inplace=True, drop=True)

    return df


def pre_calculate_data_features(df):
    # Pre-calculate time, cost and obj function value
    time_list = []
    cost_list = []
    obj1_list = []

    for i in range(len(df)):
        if df["Transportation modes"][i] == "Air":
            time_list.append(df["Air Distance (km)"][i] / df["Avg speed (km/hr)"][i])
            cost_list.append(df["Air Distance (km)"][i] * df["Cost per km"][i])
            obj1_list.append(df["Air Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)

        elif df["Transportation modes"][i] == "Road":
            time_list.append(df["Road Distance (km)"][i] / df["Avg speed (km/hr)"][i])
            cost_list.append(df["Road Distance (km)"][i] * df["Cost per km"][i])
            obj1_list.append(df["Road Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)

        else:
            time_list.append(df["Rail Distance (km)"][i] / df["Avg speed (km/hr)"][i])
            cost_list.append(df["Rail Distance (km)"][i] * df["Cost per km"][i])
            obj1_list.append(df["Rail Distance (km)"][i] * df["Co2e (g/km)"][i] / 1000)

    df["Time (hrs)"] = time_list
    df["Cost"] = cost_list
    df["obj1"] = obj1_list


def prepare_model_data(vehicles_df, distances_df):
    # Createing decision variable column in the dataframe
    df = distances_df.join(vehicles_df, how="cross")
    df["decisions"] = df["Source"] + ", " + df["Destination"] + ", " + df["Transportation modes"] + ", " + df["Vehicle_type"]

    df = clean_data_mode(df)

    pre_calculate_data_features(df)

    return df

In [497]:
def master_data_preparation(vehicles_df, demands_df):
    # master availability mapping
    master_availability = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Availability"]))

    # master demand mapping
    master_demand = {}
    for i, j in zip(
        np.array(demands_df[["Source", "Destination"]].drop_duplicates()),
        [
            0,
        ]
        * len(demands_df[["Source", "Destination"]].drop_duplicates()),
    ):
        master_demand[tuple(i)] = j

    return master_availability, master_demand


def prepare_demand_data(input_df, master_demand):
    # real/input demand mapping
    demand = {}
    for i, j in zip(np.array(input_df[["From", "To"]]), input_df["Quantity (MT)"]):
        demand[tuple(i)] = j

    for k in master_demand.keys():
        if k not in demand.keys():
            demand[k] = 0

    return demand

In [498]:
def create_optim_model(obj_df, demand_decision_df, moq_decision_df, time_decision_df, cost_decision_df2, emission_decision_df, ddf):
    # Result lists choices
    vehicle_count_choice_list = []
    material_transport_quantity_choice_list = []
    co2_emission_choice_list = []
    transportation_cost_choice_list = []
    transportation_time_choice_list = []

    # Result lists trips/return trips
    vehicle_count_trip_list = []
    material_transport_quantity_trip_list = []
    co2_emission_trip_list = []
    transportation_cost_trip_list = []
    transportation_time_trip_list = []

    ##########################################################
    # ------------------- Model Definition ------------------#
    ##########################################################
    # Create a LP minimization problem
    model = pulp.LpProblem("Co2e Optimization", pulp.LpMinimize)

    ############################################################
    # ------------------- Objective Function ------------------#
    ############################################################

    model += pulp.lpSum((obj_df['Choice decision'], obj_df['Trip decision']))

    #####################################################
    # ------------------- Constraints ------------------#
    #####################################################

    for i in tqdm(range(len(demand_decision_df))):
        if demand_decision_df['demand'][i] == 0:
            model += (pulp.LpConstraint(
                            e=pulp.lpSum((demand_decision_df['Choice decision'][i], demand_decision_df['Trip decision'][i])),
                            sense=pulp.LpConstraintEQ,
                            rhs=demand_decision_df['demand'][i],
                        ))
        elif demand_decision_df['demand'][i] >= 0:
            model += (pulp.LpConstraint(
                            e=pulp.lpSum((demand_decision_df['Choice decision'][i], demand_decision_df['Trip decision'][i])),
                            sense=pulp.LpConstraintGE,
                            rhs=demand_decision_df['demand'][i],
                        ))
        
        
    for i in tqdm(range(len(moq_decision_df))):
        if moq_decision_df['demand'][i] == 0:
            model += (pulp.LpConstraint(
                            e=pulp.lpSum((moq_decision_df['Choice decision'][i], moq_decision_df['Trip decision'][i])),
                            sense=pulp.LpConstraintEQ,
                            rhs=moq_decision_df['demand'][i],
                        ))
        elif moq_decision_df['demand'][i] >= 0:
            model += (pulp.LpConstraint(
                            e=pulp.lpSum((moq_decision_df['Choice decision'][i], moq_decision_df['Trip decision'][i])),
                            sense=pulp.LpConstraintLE,
                            rhs=moq_decision_df['demand'][i],
                        ))


    for i in tqdm(range(len(time_decision_df))):
        if time_decision_df['time_constraint'][i] >= 0:
            model += (pulp.LpConstraint(
                            e=pulp.lpSum((time_decision_df['Choice decision'][i], 2*time_decision_df['Trip decision'][i])),
                            sense=pulp.LpConstraintLE,
                            rhs=time_decision_df['time_constraint'][i],
                        ))


    for i in tqdm(range(len(cost_decision_df2))):
        if cost_decision_df2['cost_constraint'][i] >= 0:
            model += (pulp.LpConstraint(
                            e=pulp.lpSum((cost_decision_df2['Choice decision'][i], cost_decision_df2['Trip decision'][i])),
                            sense=pulp.LpConstraintLE,
                            rhs=cost_decision_df2['cost_constraint'][i],
                        ))


    # ========== Other Constraints ==========
    for i in tqdm(range(len(ddf))):
        model += (pulp.LpConstraint(
                            e=(ddf['Trip decision'][i]-1000000 * ddf['Choice decision'][i]),
                            sense=pulp.LpConstraintLE,
                            rhs=0,
                        ))

    for i in tqdm(range(len(ddf))):
        if ddf['Transportation modes'][i] != "Road":
            model += (pulp.LpConstraint(
                                e=ddf['Trip decision'][i],
                                sense=pulp.LpConstraintEQ,
                                rhs=0,
                            ))

    return model

In [499]:
def extract_output(model):
    # Extracting necessary output info. from the solver
    mt_choice = []
    mt_trip = []
    for v in model.variables():
        if v.varValue != 0:
            if v.name.split("__(")[0] == "Choice_Decision":
                mt_choice.append(
                    v.name.split("__(")[1].rstrip(")").split(",_")
                    + [
                        v.varValue,
                    ]
                )
            else:
                mt_trip.append(
                    v.name.split("__(")[1].rstrip(")").split(",_")
                    + [
                        v.varValue,
                    ]
                )

    mt_choice_output = pd.DataFrame(
        mt_choice,
        columns=[
            "Source",
            "Destination",
            "Transportation mode",
            "Vehicle type",
            "Vehicle number",
            "Count",
        ],
    )
    mt_choice_output_df = mt_choice_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
    mt_choice_output_df["S_D_M"] = (
        mt_choice_output_df["Source"] + "_" + mt_choice_output_df["Destination"] + "_" + mt_choice_output_df["Transportation mode"]
    )

    mt_trip_output = pd.DataFrame(
        mt_trip,
        columns=[
            "Source",
            "Destination",
            "Transportation mode",
            "Vehicle type",
            "Vehicle number",
            "Count",
        ],
    )
    mt_trip_output_df = mt_trip_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
    mt_trip_output_df["S_D_M"] = mt_trip_output_df["Source"] + "_" + mt_trip_output_df["Destination"] + "_" + mt_trip_output_df["Transportation mode"]
    mt_trip_output_df["Vehicle type with number"] = mt_trip_output_df["Vehicle type"] + "_" + mt_trip_output_df["Vehicle number"]

    return mt_choice_output_df, mt_trip_output_df


def create_mappings(df, distances_df, vehicles_df):
    # Apply all mapping to the output
    dist_list = []
    vals = []
    for c in distances_df.columns[2:]:
        for i in range(len(distances_df)):
            dist_list.append(distances_df["Source"][i] + "_" + distances_df["Destination"][i] + "_" + c.split(" ")[0])
            vals.append(distances_df[c][i])

    distance_mappings = dict(zip(dist_list, vals))
    quantity_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Capacity (metric tons)"]))
    emission_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Co2e (g/km)"]))
    cost_mappings = dict(zip(vehicles_df["Vehicle_type"], vehicles_df["Cost per km"]))
    time_mappings = dict(
        zip(
            (df["Source"] + "_" + df["Destination"] + "_" + df["Transportation modes"] + "_" + df["Vehicle_type"].str.replace(" ", "_")),
            df["Time (hrs)"],
        )
    )

    return {
        "distance_mappings": distance_mappings,
        "quantity_mappings": quantity_mappings,
        "emission_mappings": emission_mappings,
        "cost_mappings": cost_mappings,
        "time_mappings": time_mappings,
    }


def apply_mappings(mappings, df, t=None):
    # Get output data ready before putting into presentable format
    df["Loaded quantity"] = df["Vehicle type"].str.replace("_", " ").map(mappings["quantity_mappings"])
    df["Distance"] = df["S_D_M"].map(mappings["distance_mappings"])
    df["Emission"] = (df["Vehicle type"].str.replace("_", " ").map(mappings["emission_mappings"])) * df["Distance"] / 1000
    df["Transportation cost"] = (df["Vehicle type"].str.replace("_", " ").map(mappings["cost_mappings"])) * df["Distance"]
    df["Time (hrs)"] = (df["S_D_M"] + "_" + df["Vehicle type"]).map(mappings["time_mappings"])
    if t == ("trip" or "return" or "return trip"):
        df["Time (hrs)"] *= df["Count"]

In [500]:
def generate_output(df):
    # Pivot tables for more presentable data

    vehicle_count = pd.pivot_table(
        df,
        values="Count",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    vehicle_count["Total Vehicles in Use"] = vehicle_count.iloc[:, 2:].sum(axis=1)

    material_transport_quantity = pd.pivot_table(
        df,
        values="Loaded quantity",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    material_transport_quantity["Total Quantity Transported"] = material_transport_quantity.iloc[:, 2:].sum(axis=1)

    co2_emission = pd.pivot_table(
        df,
        values="Emission",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    co2_emission["Total co2 Emission (kg)"] = co2_emission.iloc[:, 2:].sum(axis=1)

    transportation_cost = pd.pivot_table(
        df,
        values="Transportation cost",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="sum",
    ).reset_index()
    transportation_cost["Total transportation cost"] = transportation_cost.iloc[:, 2:].sum(axis=1)

    transportation_time = pd.pivot_table(
        df,
        values="Time (hrs)",
        index=["Source", "Destination"],
        columns=["Vehicle type"],
        aggfunc="max",
    ).reset_index()
    transportation_time["Total Time (hrs)"] = transportation_time.iloc[:, 2:].sum(axis=1)

    # Some cleaning

    vehicle_count.fillna(0, inplace=True)
    material_transport_quantity.fillna(0, inplace=True)
    co2_emission.fillna(0, inplace=True)
    transportation_cost.fillna(0, inplace=True)
    transportation_time.fillna(0, inplace=True)

    return vehicle_count, material_transport_quantity, co2_emission, transportation_cost, transportation_time

In [501]:
def save_outputs(folder_path):
    os.makedirs(folder_path, exist_ok=True)
    with pd.ExcelWriter(f"{folder_path}/Vehicle_Count.xlsx") as writer1:
        vehicle_count_choice.to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        vehicle_count_trip.to_excel(writer1, sheet_name="Solution 1", index=False, startrow=1, startcol=vehicle_count_choice.shape[1] + 2)

    with pd.ExcelWriter(f"{folder_path}/Material_Transport_Quantity.xlsx") as writer2:
        material_transport_quantity_choice.to_excel(writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        material_transport_quantity_trip.to_excel(
            writer2, sheet_name="Solution 1", index=False, startrow=1, startcol=material_transport_quantity_choice.shape[1] + 2
        )

    with pd.ExcelWriter(f"{folder_path}/CO2_Emission.xlsx") as writer3:
        co2_emission_choice.to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        co2_emission_trip.to_excel(writer3, sheet_name="Solution 1", index=False, startrow=1, startcol=co2_emission_choice.shape[1] + 2)

    with pd.ExcelWriter(f"{folder_path}/Transportation_Cost.xlsx") as writer4:
        transportation_cost_choice.to_excel(writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        transportation_cost_trip.to_excel(writer4, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_cost_choice.shape[1] + 2)

    with pd.ExcelWriter(f"{folder_path}/Transportation_Time.xlsx") as writer5:
        transportation_time_choice.to_excel(writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=0)
        transportation_time_trip.to_excel(writer5, sheet_name="Solution 1", index=False, startrow=1, startcol=transportation_time_choice.shape[1] + 2)

In [502]:
def prepare_and_save_summary(folder_path, run=0):
    os.makedirs(folder_path, exist_ok=True)
    output_large_merged = mt_choice_output_df.merge(
        mt_trip_output_df[["Source", "Destination", "Transportation mode", "Vehicle type", "Vehicle number", "Count", "Time (hrs)"]],
        on=["Source", "Destination", "Transportation mode", "Vehicle type", "Vehicle number"],
        how="left",
    ).fillna(-1)
    output_large_merged["Count_y"] += 1
    output_large_merged["Count_y"] = output_large_merged[["Count_x", "Count_y"]].max(axis=1)
    output_large_merged["Time (hrs)"] = output_large_merged[["Time (hrs)_x", "Time (hrs)_y"]].max(axis=1)
    summary_large = output_large_merged[
        [
            "Source",
            "Destination",
            "Transportation mode",
            "Vehicle type",
            "Vehicle number",
            "Count_y",
            "Loaded quantity",
            "Emission",
            "Transportation cost",
            "Time (hrs)",
        ]
    ]
    summary_large.rename(
        {
            "Count_y": "Number of trips",
            "Emission": "Total CO2 Emission",
            "Transportation cost": "Total Transportation Cost",
            "Time (hrs)": "Total Transportation Time (hrs)",
        },
        axis=1,
        inplace=True,
    )
    summary_large["Vehicle number"] = summary_large["Vehicle number"].astype("int64")

    summary_large["Loaded quantity"] *= summary_large["Number of trips"]
    summary_large["Total CO2 Emission"] *= 2 * summary_large["Number of trips"]
    summary_large["Total Transportation Cost"] *= summary_large["Number of trips"]
    summary_large["Total Transportation Time (hrs)"] *= 2 * summary_large["Number of trips"] - 1

    result_df = (
        summary_large.groupby(["Source", "Destination"])
        .agg(
            {
                "Vehicle type": "count",
                "Total CO2 Emission": "sum",
                "Total Transportation Time (hrs)": "max",
                "Total Transportation Cost": "sum",
                "Loaded quantity": "sum",
            }
        )
        .reset_index()
    )
    output_short_merged = result_df.merge(input_df, left_on=["Source", "Destination"], right_on=["From", "To"])
    summary_short = output_short_merged[
        [
            "Source",
            "Destination",
            "Quantity (MT)",
            "Time (hrs)",
            "Cost",
            "Loaded quantity",
            "Total Transportation Time (hrs)",
            "Total Transportation Cost",
            "Total CO2 Emission",
        ]
    ]
    summary_short.rename(
        {
            "Quantity(MT)": "Demand",
            "Time (hrs)": "Time Constraint (hrs)",
            "Cost": "Cost Constraint",
            "Total transportation time": "Total transportation time (hrs)",
            "Loaded quantity": "Total Quantity Transported",
        },
        axis=1,
        inplace=True,
    )

    with pd.ExcelWriter(f"{folder_path}/Summary{run}.xlsx") as writer:
        summary_large.to_excel(writer, sheet_name="Summary", index=False, startrow=0, startcol=0)
        summary_short.to_excel(writer, sheet_name="Summary", index=False, startrow=summary_large.shape[0] + 3, startcol=0)

    return summary_large, summary_short

In [503]:
%%time
demand = prepare_demand_data(input_df, master_demand)
mappings = create_mappings(df0, distances_df, vehicles_df)

time = {}
for i, j in zip(np.array(input_df[["From", "To"]]), input_df["Time (hrs)"]):
    time[tuple(i)] = j

for k in master_demand.keys():
    if k not in time.keys():
        time[k] = 0
        
cost = {}
for i, j in zip(np.array(input_df[["From", "To"]]), input_df["Cost"]):
    cost[tuple(i)] = j

for k in master_demand.keys():
    if k not in cost.keys():
        cost[k] = 0
        

i1 = demand_decision_df.set_index(list(demand_decision_df[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
demand_decision_df = demand_decision_df[i1.isin(i2)]

i1 = moq_decision_df.set_index(list(moq_decision_df[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
moq_decision_df = moq_decision_df[i1.isin(i2)]

i1 = time_decision_df.set_index(list(time_decision_df[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
time_decision_df = time_decision_df[i1.isin(i2)]

i1 = cost_decision_df.set_index(list(cost_decision_df[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
cost_decision_df = cost_decision_df[i1.isin(i2)]

i1 = cost_decision_df2.set_index(list(cost_decision_df2[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
cost_decision_df2 = cost_decision_df2[i1.isin(i2)]

i1 = emission_decision_df.set_index(list(cost_decision_df2[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
emission_decision_df = emission_decision_df[i1.isin(i2)]

i1 = ddf.set_index(list(ddf[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
ddf = ddf[i1.isin(i2)]

i1 = obj_df.set_index(list(obj_df[["Source", "Destination"]])).index
i2 = input_df.set_index(list(input_df[["From", "To"]])).index
obj_df = obj_df[i1.isin(i2)]

demand_decision_df.reset_index(drop=True, inplace=True)
moq_decision_df.reset_index(drop=True, inplace=True)
time_decision_df.reset_index(drop=True, inplace=True)
cost_decision_df.reset_index(drop=True, inplace=True)
cost_decision_df2.reset_index(drop=True, inplace=True)
emission_decision_df.reset_index(drop=True, inplace=True)
ddf.reset_index(drop=True, inplace=True)
obj_df.reset_index(drop=True, inplace=True)

demand_decision_df['demand'] = demand_decision_df['S_D'].map(demand)
moq_decision_df['demand'] = moq_decision_df['S_D'].map(demand)
time_decision_df['time_constraint'] = time_decision_df['S_D'].map(time)
cost_decision_df['cost_constraint'] = cost_decision_df['S_D'].map(cost)
cost_decision_df2['cost_constraint'] = cost_decision_df2['S_D'].map(cost)
obj_df['obj1'] = obj_df['S_D'].map(cost)

model = create_optim_model(obj_df, demand_decision_df, moq_decision_df, time_decision_df, cost_decision_df2, emission_decision_df, ddf)

100%|██████████| 1140/1140 [00:00<00:00, 190202.74it/s]

CPU times: total: 1.77 s
Wall time: 1.79 s


In [504]:
%%time
model.solve(pulp.PULP_CBC_CMD())
print(pulp.LpStatus[model.status])
mt_choice_output_df, mt_trip_output_df = extract_output(model)
apply_mappings(mappings, mt_choice_output_df)
apply_mappings(mappings, mt_trip_output_df, t="return")
(
    vehicle_count_choice,
    material_transport_quantity_choice,
    co2_emission_choice,
    transportation_cost_choice,
    transportation_time_choice,
) = generate_output(mt_choice_output_df)
vehicle_count_trip, material_transport_quantity_trip, co2_emission_trip, transportation_cost_trip, transportation_time_trip = generate_output(
    mt_trip_output_df
)
save_outputs(f"../data/Output_data/v4/ans1")
summary_large, summary_short = prepare_and_save_summary(f"../data/Output_data/v4/ans1/result", 1)

Optimal
CPU times: total: 344 ms
Wall time: 1.16 s


In [505]:
summary_large

,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Number of trips,Loaded quantity,Total CO2 Emission,Total Transportation Cost,Total Transportation Time (hrs)
0,Delhi,Chennai,Road,CAR_DERIVED_VAN,0,1.0,28.0,687.870304,2528.995145,35.320484
1,Delhi,Chennai,Road,CAR_DERIVED_VAN,3,1.0,28.0,687.870304,2528.995145,35.320484
2,Delhi,Chennai,Road,CAR_DERIVED_VAN,4,1.0,28.0,687.870304,2528.995145,35.320484
3,Delhi,Chennai,Road,HGV___18_TONNES,1,1.0,36.0,797.246416,5347.570776,39.815818
4,Delhi,Chennai,Road,HGV___18_TONNES,2,1.0,36.0,797.246416,5347.570776,39.815818
5,Delhi,Chennai,Road,HGV___18_TONNES,3,1.0,36.0,797.246416,5347.570776,39.815818
6,Delhi,Chennai,Road,HGV___18_TONNES,4,1.0,36.0,797.246416,5347.570776,39.815818
7,Delhi,Kolkata,Rail,MIXED_RAIL_WCM_6,0,1.0,7000.0,219.232620,5445.920000,16.403373
8,Delhi,Kolkata,Rail,MIXED_RAIL_WCM_6,2,1.0,7000.0,219.232620,5445.920000,16.403373
9,Delhi,Mumbai,Rail,GOODS_RAIL_WAG_12,1,1.0,9000.0,224.523225,5954.700000,15.075190


In [506]:
####################################################################
# ------------------- Solve for second iteration -------------------#
####################################################################
total_cost = summary_short["Total Transportation Cost"].sum()
model += pulp.lpSum((cost_decision_df['Choice decision'], cost_decision_df['Trip decision'])) <= int(total_cost - 1)

In [507]:
total_cost, summary_short["Total CO2 Emission"].sum()

(129219.92214994426, 16285.818431251297)

In [508]:
%%time
model.solve(pulp.PULP_CBC_CMD())
# 
print(pulp.LpStatus[model.status])
mt_choice_output_df, mt_trip_output_df = extract_output(model)
apply_mappings(mappings, mt_choice_output_df)
apply_mappings(mappings, mt_trip_output_df, t="return")
(
    vehicle_count_choice,
    material_transport_quantity_choice,
    co2_emission_choice,
    transportation_cost_choice,
    transportation_time_choice,
) = generate_output(mt_choice_output_df)
vehicle_count_trip, material_transport_quantity_trip, co2_emission_trip, transportation_cost_trip, transportation_time_trip = generate_output(
    mt_trip_output_df
)
save_outputs(f"../data/Output_data/v4/ans2")
summary_large, summary_short = prepare_and_save_summary(f"../data/Output_data/v4/ans2/result", 2)

Optimal
CPU times: total: 375 ms
Wall time: 925 ms


In [509]:
###################################################################
# ------------------- Solve for third iteration -------------------#
###################################################################
total_cost = summary_short["Total Transportation Cost"].sum()
model += pulp.lpSum((cost_decision_df['Choice decision'], cost_decision_df['Trip decision'])) <= int(total_cost - 1)

In [510]:
total_cost, summary_short["Total CO2 Emission"].sum()

(129219.92214994426, 16285.818431251297)

In [511]:
summary_large

,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Number of trips,Loaded quantity,Total CO2 Emission,Total Transportation Cost,Total Transportation Time (hrs)
0,Delhi,Chennai,Road,CAR_DERIVED_VAN,1,2.0,56.0,1375.740608,5057.990290,105.961452
1,Delhi,Chennai,Road,CAR_DERIVED_VAN,3,1.0,28.0,687.870304,2528.995145,35.320484
2,Delhi,Chennai,Road,HGV___18_TONNES,1,2.0,72.0,1594.492832,10695.141551,119.447455
3,Delhi,Chennai,Road,HGV___18_TONNES,4,1.0,36.0,797.246416,5347.570776,39.815818
4,Delhi,Chennai,Road,HGV___18_TONNES,5,1.0,36.0,797.246416,5347.570776,39.815818
5,Delhi,Kolkata,Rail,MIXED_RAIL_WCM_6,0,1.0,7000.0,219.232620,5445.920000,16.403373
6,Delhi,Kolkata,Rail,MIXED_RAIL_WCM_6,2,1.0,7000.0,219.232620,5445.920000,16.403373
7,Delhi,Mumbai,Rail,GOODS_RAIL_WAG_12,1,1.0,9000.0,224.523225,5954.700000,15.075190
8,Kolkata,Chennai,Air,CARGO_PLANE,0,1.0,100.0,1604.017991,13553.000000,2.673176
9,Kolkata,Chennai,Air,CARGO_PLANE,1,1.0,100.0,1604.017991,13553.000000,2.673176


In [512]:
%%time
model.solve(pulp.PULP_CBC_CMD())
print(pulp.LpStatus[model.status])
mt_choice_output_df, mt_trip_output_df = extract_output(model)
apply_mappings(mappings, mt_choice_output_df)
apply_mappings(mappings, mt_trip_output_df, t="return")
(
    vehicle_count_choice,
    material_transport_quantity_choice,
    co2_emission_choice,
    transportation_cost_choice,
    transportation_time_choice,
) = generate_output(mt_choice_output_df)
vehicle_count_trip, material_transport_quantity_trip, co2_emission_trip, transportation_cost_trip, transportation_time_trip = generate_output(
    mt_trip_output_df
)
save_outputs(f"../data/Output_data/v4/ans3")
summary_large, summary_short = prepare_and_save_summary(f"../data/Output_data/v4/ans3/result", 3)

Optimal
CPU times: total: 359 ms
Wall time: 829 ms


In [513]:
total_cost = summary_short["Total Transportation Cost"].sum()
total_cost, summary_short["Total CO2 Emission"].sum()

(129219.92214994426, 16285.818431251297)

In [514]:
summary_large

,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Number of trips,Loaded quantity,Total CO2 Emission,Total Transportation Cost,Total Transportation Time (hrs)
0,Delhi,Chennai,Road,CAR_DERIVED_VAN,0,1.0,28.0,687.870304,2528.995145,35.320484
1,Delhi,Chennai,Road,CAR_DERIVED_VAN,1,1.0,28.0,687.870304,2528.995145,35.320484
2,Delhi,Chennai,Road,CAR_DERIVED_VAN,2,1.0,28.0,687.870304,2528.995145,35.320484
3,Delhi,Chennai,Road,HGV___18_TONNES,2,1.0,36.0,797.246416,5347.570776,39.815818
4,Delhi,Chennai,Road,HGV___18_TONNES,3,1.0,36.0,797.246416,5347.570776,39.815818
5,Delhi,Chennai,Road,HGV___18_TONNES,5,1.0,36.0,797.246416,5347.570776,39.815818
6,Delhi,Chennai,Road,HGV___18_TONNES,6,1.0,36.0,797.246416,5347.570776,39.815818
7,Delhi,Kolkata,Rail,MIXED_RAIL_WCM_6,0,1.0,7000.0,219.232620,5445.920000,16.403373
8,Delhi,Kolkata,Rail,MIXED_RAIL_WCM_6,2,1.0,7000.0,219.232620,5445.920000,16.403373
9,Delhi,Mumbai,Rail,GOODS_RAIL_WAG_12,1,1.0,9000.0,224.523225,5954.700000,15.075190


In [515]:
del (
    model,
    mt_choice_output_df,
    mt_trip_output_df,
    mappings,
    vehicle_count_choice,
    material_transport_quantity_choice,
    co2_emission_choice,
    transportation_cost_choice,
    transportation_time_choice,
    vehicle_count_trip,
    material_transport_quantity_trip,
    co2_emission_trip,
    transportation_cost_trip,
    transportation_time_trip,
    summary_large,
    summary_short,
    total_cost,
)
gc.collect()

93